In [ ]:
##### pacotes necessários
!pip install PyMuPDF
!pip install PyPDF2
!pip install spacy --upgrade
!pip install xgboost
!pip install transformers
!pip install sentence_transformers

## download pipeline em português
!python -m spacy download pt_core_news_lg

import PyPDF2
import numpy as np
import pandas as pd
import fitz
import re
import nltk
import matplotlib.pyplot as plt
import collections
import spacy
import string
import torch
import seaborn as sns

nltk.download('stopwords')
nltk.download('punkt')
from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from spacy import displacy
from spacy.lang.pt.examples import sentences
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from transformers import AutoTokenizer  
from transformers import AutoModelForPreTraining  
from transformers import AutoModel
from transformers import BertTokenizer
from torch import nn
from transformers import BertModel
from torch.optim import Adam
from tqdm import tqdm
from xgboost import XGBClassifier
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist as scipy_cdist
from pathlib import Path 
%matplotlib inline
from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from pandas.core.frame import DataFrame
from wordcloud import WordCloud
from nltk.tokenize import word_tokenize
from spacy import displacy
from spacy.lang.pt.examples import sentences

In [ ]:
##criação tag - pipeline Spacy
tag = spacy.load("pt_core_news_lg")

In [ ]:
## pasta com pdfs - montar drive antes
# para montar o drive, basta clicar em arquivos e depois no link do google drive
# as instruções são claras a partir daí 
path = "/content/drive/MyDrive/..."
%cd "/content/drive/MyDrive/..."

In [ ]:
## verificação - pdfs contidos na pasta
list_files = listdir(path)
list_files

In [ ]:
## nome dos pdfs
files = [join("/content/drive/MyDrive/...", f) for f in list_files]
files

In [ ]:
##loop para leitura dos textos
textos = []
for f in files:
  with fitz.open(f) as doc:
    text = ""
    for page in doc:
        text += page.get_text()
  textos.append(text)

In [ ]:
## verificação - tamanho do objeto textos
len(textos)

In [ ]:
## verificação - objeto textos
textos

In [ ]:
##### limpeza básica
## removendo espaços em branco
textos = [f.strip() for f in textos]
textos = [" ".join(f.split()) for f in textos]

In [ ]:
## lowercase, remoção caracteres especiais, palavras pequenas e grandes demais, 
## números desnecessários
textos = [f.lower() for f in textos]
textos = [re.sub(':', ' ', f) for f in textos]
textos = [f.strip('.,') for f in textos]
textos = [f.replace('.',' ') for f in textos]
textos = [f.replace('-',' ') for f in textos]
textos = [f.replace('/',' ') for f in textos]
textos = [re.sub(r'[0-9]+', '', f) for f in textos]
textos = [f.replace('º',' ') for f in textos]
textos = [f.replace('+',' ') for f in textos]
textos = [f.replace('*',' ') for f in textos]
textos = [f.replace('#',' ') for f in textos]
textos = [f.replace('@',' ') for f in textos]
textos = [f.replace('&',' ') for f in textos]
textos = [f.replace('–',' ') for f in textos]
textos = [f.replace(',',' ') for f in textos]
textos = [f.replace(';',' ') for f in textos]
textos = [f.replace(')',' ') for f in textos]
textos = [f.replace('(',' ') for f in textos]
textos = [f.replace('_',' ') for f in textos]
textos = [f.replace(']',' ') for f in textos]
textos = [f.replace('{',' ') for f in textos]
textos = [f.replace('}',' ') for f in textos]
textos = [f.replace('[',' ') for f in textos]
textos = [f.replace('%',' ') for f in textos]
textos = [f.replace('§',' ') for f in textos]
textos = [f.replace('“',' ') for f in textos]
textos = [f.replace('”',' ') for f in textos]
textos = [f.replace('ª',' ') for f in textos]
textos = [f.replace('°',' ') for f in textos]
textos = [re.sub(r'\b\w{1,3}\b', ' ', f) for f in textos]
textos = [re.sub(r'\b\w{15,}\b', ' ', f) for f in textos]
textos = [f.strip() for f in textos]
textos = [" ".join(f.split()) for f in textos]

In [ ]:
## verificação - objeto textos
textos

In [ ]:
## combinação - stopwords NLTK com outras palavras de parada
stop_words = set(stopwords.words('portuguese'))
palavras_retirar = ["página","autenticacao","documento","e","podem","que","de",
                    "abr","jan","mar","mai","jun","ago","set","out","nov","dez",
                    "fev","jul","a","o","dele","ingresso","geral","dela",
                    "ele","ela","as", "os", "à", "mais", "nos","por","senhora",
                    "às","na","no","porque","porquê","por quê","por que", 
                    "qual","quais","isso","se","por","ainda","ou","das", 
                    "dos","só","em","com","esta","está","da","do","dar",
                    "dá","para","fins","sim","não","seu","sua","meu", 
                    "nosso","nossa","esse","este","aquele","aquela","aquilo",
                    "temos","base","como","quando","aqueles","aquelas","nossos",
                    "nossas","meus","qual","firme","pode","causar","caput",
                    "casos","caso","inteiro","teor", "portal","eletrônico",
                    "institui","agravo","regimental","assinado","chaves",
                    "ementa","acessado","portal","número","direta","para",
                    "como","endereço","http","digitalmente","sido","senhor"]
palavras_retirar = set(palavras_retirar)
stop_words.update(palavras_retirar)

In [ ]:
## remoção de stopwords
textos = [word_tokenize(f) for f in textos]
textos_limpos = []
for f in textos:
  text = [word for word in f if word not in stop_words]
  text = " ".join(text)
  textos_limpos.append(text)

In [ ]:
## verifição - objeto textos_limpos
textos_limpos

In [ ]:
## opcional - truncar textos_limpos (10000 palavras, por exemplo)
#se for uma opção, usar o objeto textos_trunc a partir daqui
textos_trunc = [" ".join(f.split()[:10000]) for f in textos_limpos]

In [ ]:
## dataframe inicial - usar textos_limpos ou textos_trunc
list_files_limpo = [re.sub('.pdf', '', f) for f in list_files]
list_files_limpo = [re.sub('_', '', f) for f in list_files_limpo]
data = pd.DataFrame({'ID':list_files_limpo, 'textos':textos_trunc})

In [ ]:
## verificação - objeto data
data

In [ ]:
## verificação - texto 1
data['textos'][0]

In [ ]:
## verificação - qtd caracteres texto 1
len(data['textos'][0])

In [ ]:
## opcional - pos tag para manter verbos e nomes
## ou outras classes de interesse, como advérbios, adjetivos e etc
for i in range(len(data)):
  doc = data['textos'][i]
  doc = tag(doc)
  verbos_nomes = []
  for token in doc:
    if token.pos_ == 'VERB' or token.pos_ == 'NOUN' or token.pos_ == 'PROPN':
        verbos_nomes.append(token.lemma_)

  text = " ".join(verbos_nomes)
  data['textos'][i] = text

In [ ]:
## verificação - qtd caracteres texto 1
len(data['textos'][0])

In [ ]:
## leitura dos metadados - arquivo excel
## metadados obtidos no hotsite da Agenda 2030 no STF

# pasta do google drive onde o arquivo xlsx se encontra
%cd "/content/drive/MyDrive/..."

# trocar nome do arquivo
ODS = pd.read_excel("nome_arquivo_metadados.xlsx") 

In [ ]:
## verificação - objeto ODS
print(np.unique(ODS['DESC_ODS']))

In [ ]:
## pivoteamento (objeto ODS é long) + limpeza colunas inúteis
ODS_bin = ODS.drop(columns='DESC_ODS').join(pd.get_dummies(ODS['DESC_ODS']))

# exemplo de limpeza - coluna inútil
# repetir linha de código .drop para todas as colunas inúteis
ODS_bin.drop("NUMERO", inplace=True, axis=1)

# groupby - ID
ODS_bin = ODS_bin.groupby(["ID"]).sum()

# rename - fazer isso para todos os ODS que aparecem na saída da célula anterior
# os exemplos abaixo contemplam apenas alguns ODS
ODS_bin.rename(columns={'ODS 10 - Redução das Desigualdades': 'ODS_10'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 11 - Cidades e Comunidades Sustentáveis': 'ODS_11'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 16 - Paz, Justiça e Instituições Eficazes': 'ODS_16'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 17 - Parcerias e Meios de Implementação': 'ODS_17'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 5 - Igualdade de Gênero': 'ODS_5'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 8 - Trabalho Decente e Crescimento Econômico': 'ODS_8'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 3 - Saúde e Bem-Estar': 'ODS_3'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 1 - Erradicação da Pobreza': 'ODS_1'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 12 - Consumo e Produção Responsáveis': 'ODS_12'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 15 - Vida Terrestre': 'ODS_15'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 2 - Fome Zero e Agricultura Sustentável': 'ODS_2'}, 
               inplace = True)
ODS_bin.rename(columns={'ODS 4 - Educação de Qualidade': 'ODS_4'}, 
               inplace = True)

In [ ]:
## verificação - objeto ODS_bin
ODS_bin

In [ ]:
## criação do objeto data final - merge
data_final = pd.merge(data,ODS_bin,on='ID',how='right')

In [ ]:
## lowercase novamente - PROPN spacy altera a letra inicial
data_final['textos'] = [f.lower() for f in data_final['textos']]

In [ ]:
## verificação - objeto data_final
data_final

In [ ]:
##### visualizações
## criando uma cópia do objeto data_final
data_final_viz = data_final.copy(deep=True)

In [ ]:
## incluindo uma coluna com os tokens de cada texto
data_final_viz['tokens_textos'] = data_final_viz.apply(lambda row: nltk.word_tokenize(row['textos']), axis=1)

In [ ]:
## verificação - objeto data_final_viz
data_final_viz

In [ ]:
## verificação - tokens do texto 1
data_final_viz["tokens_textos"][0]

In [ ]:
## frequência de palavras - texto 1
contagem_palavras = collections.Counter(data_final_viz["tokens_textos"][0])
print("palavras mais comuns:", contagem_palavras.most_common(10))

In [ ]:
## transformação em data frame - frequência de palavras texto 1
contagem_palavras = pd.DataFrame(contagem_palavras.most_common(25),
                                     columns=["palavra", "freq"])

In [ ]:
## verificação - objeto contagem_palavras
print(contagem_palavras)

In [ ]:
## gráfico de palavras mais frequentes
# set tamanho da figura
fig, ax = plt.subplots(figsize=(12, 8))
# horizontal bar plot
contagem_palavras.sort_values(by='freq').plot.barh(x="palavra", y="freq", ax=ax)
# título
plt.title("palavras mais comuns no texto")
plt.show()

In [ ]:
## nuvem de palavras
word_cloud = WordCloud(collocations = False, background_color = 'white').generate(data_final_viz["textos"][0])
plt.figure(figsize=(16, 13), dpi = 400)
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis()
plt.show()

In [ ]:
##### contagem de palavras-chave
## leitura do texto bruto
textos_brutos = []
for f in files:
  with fitz.open(f) as doc:
    text = ""
    for page in doc:
        text += page.get_text()
  textos_brutos.append(text)

In [ ]:
## limpeza mínima
textos_brutos = [f.lower() for f in textos_brutos]
textos_brutos = [f.strip() for f in textos_brutos]
textos_brutos = [" ".join(f.split()) for f in textos_brutos]

In [ ]:
## verificação - objeto textos brutos
textos_brutos

In [ ]:
## palavras-chave de exemplo
palavras_chave_1 = ["saúde", "bem-estar", "mortalidade materna", "mortalidade infantil", "mortalidade neonatal", "aids", "hiv",
                  "tuberculose", "malária", "hepatite", "saúde mental", "câncer", "diabetes", "doenças respiratórias"]
palavras_chave_2 = ["suicídio", "álcool", "drogas", "entorpecentes", "acidentes de trânsito", "saúde mental", "saúde reprodutiva",
                  "saúde essencial", "saúde pública", "doença", "medicamentos", "vacina","ministro","ministro presidente","suspensão","plenário","centro"]
palavras_chave = [palavras_chave_1,palavras_chave_2]

In [ ]:
## loop para contar ocorrências
contagem = []
for p in palavras_chave:
  cont = [sum(f.count(x) for x in p) for f in textos_brutos]
  contagem.append(cont)
contagem = pd.DataFrame(contagem).transpose()
contagem.columns = ['ODS_TESTE_1', 'ODS__TESTE_2']

In [ ]:
## verificação - objeto contagem
contagem

In [ ]:
##### classificação via redes neurais
## foco no ODS 16 - groupby 
data_final.groupby(['ODS_16']).size().plot.bar()

In [ ]:
## opcional - tamanho dos textos
data_final['textos'] = data_final['textos'].astype(str)
data_final['length'] = data_final.textos.str.len()
data_final = data_final[data_final.length > 10]
data_final.drop("length", inplace=True, axis=1)

In [ ]:
## BERT
model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased', do_lower_case=False)

In [ ]:
## criação dos labels
labels = {0:0,
          1:1
          }

In [ ]:
## classe Dataset - Pytorch
class Dataset(torch.utils.data.Dataset):

    def __init__(self, data_final):

        self.labels = [labels[label] for label in data_final['ODS_16']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 32, truncation=True,
                                return_tensors="pt") for text in data_final['textos']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y


In [ ]:
## conjunto de treino, teste e validação - 25, 35 e 40 são limites de exemplo
## a montagem das bases de treino, validação e testes pode ser feita de forma 
## aleatório também. 
df_train = data_final.iloc[0:25]
df_val = data_final.iloc[25:35]
df_test = data_final.iloc[35:40]

In [ ]:
## verificação - tamanho dos conjuntos
print(len(df_train),len(df_val), len(df_test))

In [ ]:
## classificador
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.4):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('neuralmind/bert-base-portuguese-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
## execução da rede
def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label)
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label)
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 10
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

In [ ]:
## avaliação da rede
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    global total_acc_test
    total_acc_test = 0
    global lista
    lista = []
    global lista_pred
    lista_pred = []
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              
              lista_pred.append(output.argmax(dim=1))
              acc = (output.argmax(dim=1) == test_label).sum().item()
              lista.append(acc)
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
    
evaluate(model, df_test)

In [ ]:
## acertos da rede
total_acc_test

In [ ]:
## valores preditos
lista_pred = [f.item() for f in lista_pred]

In [ ]:
##### classificação via XGboost
## bag of words
vectorizer = CountVectorizer(decode_error='replace', encoding='utf-8')
bow_transformer_train = vectorizer.fit(df_train["textos"].values.astype('U'))
bow_transformer_test = vectorizer.fit(df_test["textos"].values.astype('U'))
bow_transformer_val = vectorizer.fit(df_val["textos"].values.astype('U'))

In [ ]:
## convertendo strings em vetores usando BoW
textos_bow_train = bow_transformer_train.transform(df_train["textos"].values.astype('U'))
textos_bow_test = bow_transformer_test.transform(df_test["textos"].values.astype('U'))
textos_bow_val = bow_transformer_val.transform(df_val["textos"].values.astype('U'))

In [ ]:
## shape textos_bow
print(f"Shape: {textos_bow_train.shape}")
print(f"Quantidade de ocorrências diferentes de zero: {textos_bow_train.nnz}")

In [ ]:
## TF-IDF
tfidf_transformer_train = TfidfTransformer().fit(textos_bow_train)
textos_tfidf_train = tfidf_transformer_train.transform(textos_bow_train)

tfidf_transformer_test = TfidfTransformer().fit(textos_bow_test)
textos_tfidf_test = tfidf_transformer_test.transform(textos_bow_test)

tfidf_transformer_val = TfidfTransformer().fit(textos_bow_val)
textos_tfidf_val = tfidf_transformer_val.transform(textos_bow_val)

In [ ]:
## shape textos_tfidf
print(textos_tfidf_train.shape)

In [ ]:
## split textos_tfidf em conjuntos de treino e testes
label_train = df_train['ODS_16']
label_val = df_val['ODS_16']
label_test = df_test['ODS_16']

In [ ]:
## visualização - objeto label_train
label_train

In [ ]:
## instância do modelo
clf = XGBClassifier()
clf.fit(textos_tfidf_train, label_train)

In [ ]:
## predicões - conjunto de treino
predict_train = clf.predict(textos_tfidf_train)

print(
    f"Acurácia - Conjunto de Treino: {metrics.accuracy_score(label_train, predict_train):0.3f}"
)

In [ ]:
## exemplo de predição - texto específico
print(
    "predição:",
    clf.predict(
        tfidf_transformer_train.transform(bow_transformer_train.transform([data_final["textos"][9]]))
    )[0],
)
print("classe verdadeira:", data_final["ODS_16"][9])

In [ ]:
## acurácia - conjunto de teste
label_predictions = clf.predict(textos_tfidf_test)
print(f"Acurácia do modelo: {metrics.accuracy_score(label_test, label_predictions):0.3f}")

In [ ]:
## examinando o objeto - label_predictions
label_predictions

In [ ]:
##### similaridade entre textos
## TF IDF - Embedding
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=50, max_df=0.50,
                             stop_words=stopwords.words('portuguese'), 
                             analyzer='word',
                             ngram_range=(1, 1), 
                             lowercase=True, 
                             use_idf=True)
X = vectorizer.fit_transform(data_final['textos']).toarray()

In [ ]:
## similaridade por cosseno usando TF IDF
idx_1 = data_final.index
idx_2 = data_final.index
simil = 1 - scipy_cdist(X[idx_2], 
                               X[idx_1], 'cosine')
simil = np.around(simil, decimals=3)

cos_sims_df = pd.DataFrame(data=simil,
                       columns=data_final.ID.loc[idx_1].tolist(),
                       index=data_final.ID.loc[idx_2].tolist())
cos_sims_df.sort_index(axis=1, inplace=True) 

In [ ]:
## avaliação - objeto cos_sims_df
cos_sims_df